In [1]:
import sys
import torch
import torchvision.transforms as T
from IPython.display import Audio
import librosa

from scipy import signal
from scipy.signal import ShortTimeFFT
from scipy.signal.windows import gaussian
from scipy.signal import stft, istft

In [2]:
sample_rate=22050
n_fft=512
root = '../Data'


In [ ]:
from src.Dataset import AudioMNIST
import src.custom_transforms as CT
# Test the dataset
try:
    transforms = [
        CT.TrimSilence(5),
        CT.TimeStretchFixLength(sample_rate)
    ]
    dataset = AudioMNIST(target_sample_rate=sample_rate, transform=T.Compose(transforms))
    mel_spec, label = dataset[0]
    print("Success!")
    print("Mel spectrogram shape:", mel_spec.shape)
    print("Label:", label)
    print(f"Total samples: {len(dataset)}")
    print(f"Value range: [{mel_spec.min():.3f}, {mel_spec.max():.3f}]")
    
    # Test for NaN/Inf values
    print("Contains NaN:", torch.isnan(mel_spec).any())
    print("Contains Inf:", torch.isinf(mel_spec).any())
    
    # Additional statistics
    print(f"Mean value: {mel_spec.mean():.3f}")
    print(f"Std value: {mel_spec.std():.3f}")
except Exception as e:
    print(f"Error: {str(e)}")

In [ ]:
import numpy as np
Zxx , _= dataset[15] 

Zxxx = np.abs(Zxx)
Zxxx = np.atleast_3d(Zxxx).transpose(2,0,1)
        # convert to decibel
Zxxx = librosa.amplitude_to_db(Zxxx, ref = np.max)
import matplotlib.pyplot as plt
librosa.display.specshow(Zxxx[0])
plt.title("Real")

In [1]:
from torchsummary import summary
from src.Classifier import LightningAudioClassifier

model = LightningAudioClassifier()


summary(model, (2, 128, 173),device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 173]             608
       BatchNorm2d-2         [-1, 32, 128, 173]              64
              ReLU-3         [-1, 32, 128, 173]               0
         MaxPool2d-4           [-1, 32, 64, 86]               0
           Dropout-5           [-1, 32, 64, 86]               0
            Conv2d-6           [-1, 64, 64, 86]          18,496
       BatchNorm2d-7           [-1, 64, 64, 86]             128
              ReLU-8           [-1, 64, 64, 86]               0
         MaxPool2d-9           [-1, 64, 32, 43]               0
          Dropout-10           [-1, 64, 32, 43]               0
           Conv2d-11          [-1, 128, 32, 43]          73,856
      BatchNorm2d-12          [-1, 128, 32, 43]             256
             ReLU-13          [-1, 128, 32, 43]               0
        MaxPool2d-14          [-1, 128,